In [28]:
import math
import numpy as np 
from random import randint
import nelinOpt as no
from Matrica import Matrica, JedinicnaMatrica


## Box

In [69]:
def box(x0, xd, xg, func, ogranicenja, alfa=2.0, epsilon=1e-6, ispis=False):

    for i in range(x0.br_stup):
        if not (x0[0][i] >= xd and x0[0][i] <= xg): 
            raise Exception("Pocetna tocka je krivo zadana. Ne postuje eksplicitna ogranicenja.")
            return
    if not provjeri(x0, ogranicenja): 
            raise Exception("Pocetna tocka je krivo zadana. Ne postuje implicitna ogranicenja.")
            return
    
    n = x0.br_stup
    # print(type(x0))
    xc = x0.copy()
    # print(type(xc))
    tocke = [x0]

    #generiranje skupa 2n tocaka
    for t in range(2*n):
        print(type(xc))
        xi = []
        for i in range(n):
            r = randint(0, 1)
            xi.append(xd + r * (xg - xd))
        tocke.append(Matrica(1, n, [xi]))
        while True:
            print(type(xc))
            if provjeri(tocke[t], ogranicenja): break
            tocke[t] = 0.5 * (tocke[t] + xc)

        #novi centroid
        print(xc)
        xc = centroid(tocke, n)
        print(xc)
    
    while True:
        #pronalazak h i h2
        h = no.index_max(tocke, func, ispis)

        tocke_bez_h = tocke.copy()
        tocke_bez_h.pop(h)

        h2 = no.index_max(tocke_bez_h, func, ispis)
        xc = centroid(tocke_bez_h, n)

        xr = (1 + alfa) * xc - alfa * tocke[h]

        for i in range(n):
            if xr[0][i] < xd: xr[0][i] = xd
            if xr[0][i] > xg: xr[0][i] = xg
        
        while True:
            if provjeri(xr, ogranicenja): break
            xr = 0.5 * (xr + xc)
        
        if func.vrijednost(xr) > func.vrijednost(tocke[h2]):
            xr = 0.5 * (xr + xc)
        
        tocke[h] = xr 
        if uvjet_zaustavljanja(tocke[h], xc, epsilon): return xc    

def provjeri(x, ogranicenja):
    for o in ogranicenja:
        if o(x) < 0: return False
    return True

def centroid(tocke, n):
    xc = Matrica(1, n)
    for xi in tocke:
        xc += xi
    for i in range(n):
        xc[0][i] /= len(tocke) 
    return xc

def uvjet_zaustavljanja(xh, xc, e):
    for i in range(len(xh)):
        if abs(xh[0][i] - xc[0][i]) < e: return True
    return False

## Transformacija u problem bez ogranicenja na mjesoviti nacin

In [61]:


def transformacija(x0, func, ogranicenja, ogr_jednakosti=None, t=1., epsilon=1e-6):
    x = x0.copy
    transformirana_fja = Transformirana_fja(func, ogranicenja, ogr_jednakosti, t, epsilon)
    while True:
        optimized_value = no.hooke_jeeves(x, transformirana_fja)
        transformirana_fja.t *= 10

        if uvjet_zaustavljanja_transf(x, optimized_value):
            trenutni_t = transformirana_fja.t
            cnt = int(math.log10(t)) + 1
            return optimized_value, cnt

        x = optimized_value

class Transformirana_fja:

    def __init__(self, func, ogranicenja, ogr_jednakosti, t=1.0, epsilon=1e-6):
        self.func = func
        self.ogranicenja = ogranicenja
        self.ogr_jednakosti = ogr_jednakosti
        self.t = t
        self.epsilon = epsilon 
    
    def vrijednost(self, x):
        x = x0.copy()
        fx = func.vrijednost(x)
        
        # ogranicenja nejednakosti
        gx = 0
        for o in ogranicenja:
            ox = o(x)
            if ox < 0: return 1234567890
            else: gx += math.log(ox)
        gx /= t
        
        #ogranicenja jednakosti
        if not ogr_jednakosti != None:
            hx = t * (x[0][1] - 1)**2

        return fx - gx + hx 

    
    def uvjet_zaustavljanja_transf(x, optimized_value):
        for i in range(x):
            if abs(x[0][i] - optimized_value[0][i]) > epsilon:
                return False
        return True

## Newton-Raphson

In [46]:
def newton_raphson(x0, func, grad, hess, zl_rez=False, epsilon=1e-6, ispis=False):
    n = x0.br_stup
    x = x0.copy()
    cnt = 0
    f_tmp = func.vrijednost(x)

    while True:
        gradijent = Matrica(1, n, [grad(x)])
        if ispis: print("gradijent: ", gradijent)
        hessian = Matrica(1, n, hess(x))
        if ispis: print("hessian: ", hessian)
        v = hessian * (gradijent.transponiraj())

        if norm(v) <= epsilon:
            break

        if cnt >= 100:
            print("Ne konvergira.")
            print("Broj iteracija: ", cnt)
            return x

        if zl_rez:
            l_func = no.Funkcija(lambda l: func.vrijednost(x + l * gradijent))
            faktor = no.minimum(Matrica(1, 1, [[0.]]), l_func, gradijent, ispis)
        else:
            faktor = -1
        
        x += faktor * gradijent
        fx = func.vrijednost(x)

        if fx != f_tmp: cnt=0
        else: cnt += 1
        f_tmp = fx

    return x

## Gradijentni spust

In [42]:
def grad_desc(x0, func, grad, zl_rez=False, epsilon=1e-6, ispis=False):
    n = x0.br_stup
    x = x0.copy()
    cnt = 0
    f_tmp = func.vrijednost(x)

    while True:
        gradijent = Matrica(1, n, [grad(x)])
        if norm(gradijent) < epsilon: 
            print("Broj iteracija: ", cnt)
            break
        # v = -1 * gradient

        if cnt >= 100:
            print("Ne konvergira.")
            print("Broj iteracija: ", cnt)
            return x
        
        if zl_rez:
            l_func = no.Funkcija(lambda l: func.vrijednost(x + l * gradijent))
            faktor = no.minimum(Matrica(1, 1, [[0.]]), l_func, gradijent, ispis)
        else:
            faktor = -1
    
        x += faktor * gradijent
        fx = func.vrijednost(x)

        if fx != f_tmp: cnt=0
        else: cnt += 1
        f_tmp = fx 

    return x


def norm(x):
    squared = [xi**2 for xi in x[0]]
    sum_of_squares = sum(squared)
    return math.sqrt(sum_of_squares)


## Zadatci

1. Primijenite postupak gradijentnog spusta na funkciju 3, uz i bez određivanja optimalnog iznosa
koraka. Što možete zaključiti iz rezultata? 

In [6]:
x0 = Matrica(1, 2, [[0, 0]])
print(grad_desc(x0, no.Funkcija(no.f3), no.gf3, zl_rez=False))
print("--------------")
print(grad_desc(x0, no.Funkcija(no.f3), no.gf3, zl_rez=True))
print("--------------")

Ne konvergira.
Broj iteracija:  100
[[0. 0.]]
--------------
Broj iteracija:  0
[[ 2.00000025 -3.00000038]]
--------------


2. Primijenite postupak gradijentnog spusta i Newton-Raphsonov postupak na funkcije 1 i 2 s
određivanjem optimalnog iznosa koraka. Kako se Newton-Raphsonov postupak ponaša na ovim
funkcijama? Ispišite broj izračuna funkcije, gradijenta i Hesseove matrice. 

In [14]:
x1 = Matrica(1, 2, [[-1.9, 2]])
# print(grad_desc(x1, no.Funkcija(no.f1), no.gf1, zl_rez=True))
print(newton_raphson(x1, no.Funkcija(no.f1), no.gf1, no.hf1, zl_rez=True))
x2 = Matrica(1, 2, [[0.1, 0.3]])
print(grad_desc(x2, no.Funkcija(no.f2), no.gf2, zl_rez=True))
print(newton_raphson(x2, no.Funkcija(no.f2), no.gf2, no.hf2, zl_rez=True))


Ne konvergira.
Broj iteracija:  100
[[1.00000348 1.00000697]]
Broj iteracija:  0
[[3.99999967 2.00000005]]
[[3.99999953 1.99999978]]


3. Primijenite postupak po Boxu na funkcije 1 i 2 uz implicitna ograničenja: (x2-x1 >= 0), (2-x1 >= 0) i
eksplicitna ograničenja prema kojima su sve varijable u intervalu [-100, 100]. Mijenja li se položaj
optimuma uz nametnuta ograničenja?

In [70]:
x1 = Matrica(1, 2, [[-1.9, 2]])
x2 = Matrica(1, 2, [[0.1, 0.3]])

print(box(x1, -100, 100, no.Funkcija(no.f1), [no.o1, no.o2, no.o31, no.o32, no.o41, no.o42]))
print(box(x2, -100, 100, no.Funkcija(no.f2), [no.o1, no.o2, no.o31, no.o32, no.o41, no.o42]))

<class 'Matrica.Matrica'>
<class 'Matrica.Matrica'>
[[-1.9  2. ]]
[[-50.95  51.  ]]
<class 'Matrica.Matrica'>
<class 'Matrica.Matrica'>
[[-50.95  51.  ]]
[[-0.63333333 67.33333333]]
<class 'Matrica.Matrica'>
<class 'Matrica.Matrica'>
<class 'Matrica.Matrica'>
<class 'Matrica.Matrica'>
<class 'Matrica.Matrica'>
<class 'Matrica.Matrica'>
<class 'Matrica.Matrica'>
<class 'Matrica.Matrica'>
[[-0.63333333 67.33333333]]
[[-0.24023438 67.4609375 ]]
<class 'Matrica.Matrica'>
<class 'Matrica.Matrica'>
<class 'Matrica.Matrica'>
<class 'Matrica.Matrica'>
<class 'Matrica.Matrica'>
<class 'Matrica.Matrica'>
<class 'Matrica.Matrica'>
<class 'Matrica.Matrica'>
[[-0.24023438 67.4609375 ]]
[[-39.92698364  27.56262207]]


NameError: name 'x' is not defined

4. Primijenite postupak transformacije u problem bez ograničenja na funkcije 1 i 2 s ograničenjima iz
prethodnog zadatka (zanemarite eksplicitna ograničenja). Novodobiveni problem optimizacije bez
ograničenja minimizirajte koristeći postupak Hooke-Jeeves ili postupak simpleksa po Nelderu i
Meadu. Može li se uz zadanu početnu točku pronaći optimalno rješenje problema s ograničenjima?
Ako ne, probajte odabrati početnu točku iz koje je moguće pronaći rješenje. 

5. Za funkciju 4 s ograničenjima (3-x1-x2>=0), (3+1.5*x1-x2>=0) i (x2-1=0) probajte pronaći
minimum koristeći postupak transformacije u problem bez ograničenja (također koristite HookeJeeves ili postupak simpleksa po Nelderu i Meadu za minimizaciju). Probajte kao početnu točku
postaviti neku točku koja ne zadovoljava ograničenja nejednakosti (primjerice točku (5,5)) te
pomoću postupka pronalaženja unutarnje točke odredite drugu točku koja zadovoljava ograničenja
nejednakosti te ju iskoristite kao početnu točku za postupak minimizacije. 

## Provjere

In [30]:
tf = Transformirana_fja(3, 4, 5)
tf.t *= 10
print(tf.t)

10.0


In [24]:
p = 1.
print(p)

1.0


In [22]:
g = [5, 6]
d = Matrica(1, 2, [g])
d1 = Matrica(1, 2, [[3, 4]])
d1 += 2*d
print(d1)
norm(d)

__init__ elementi:  [[5, 6]]
__init__ elementi:  [[3, 4]]
[[13. 16.]]


7.810249675906654

In [14]:
x = [1, 2, 3]
norm(x)

3.7416573867739413

In [15]:
n = 2
tocke = [
    Matrica(1, n, [[10, 13]]),
    Matrica(1, n, [[12, 16]]),
    Matrica(1, n, [[14, 13]])
]
xc = Matrica(1, n)
for xi in tocke:
    xc += xi
for i in range(n):
    xc[0][i] /= len(tocke) 
xcopy = xc.copy()
print(xcopy)

__init__ elementi:  [[10, 13]]
__init__ elementi:  [[12, 16]]
__init__ elementi:  [[14, 13]]
__init__ elementi:  [[12. 14.]]
[[12. 14.]]


In [8]:
np.zeros((2, 3))

array([[0., 0., 0.],
       [0., 0., 0.]])